# Generate exploration problem set (Chat history)

Find a file, write a question, and create a path

AutoGen Version 0.2.2

In [1]:
import autogen
from autogen import AssistantAgent, UserProxyAgent

In [2]:

import glob
import json
import os
import pdb
import random
import re
import sys
from typing import List

from termcolor import colored

from gpt_utils import num_tokens, reply

sys.path.append('../')
from utils import list_files  # SPM's utils

MODEL_NAME = "gpt-35-turbo"
MAX_COMMAND_STEPS = 100
NEED_VERIFY = False
REPOS_ROOT = os.path.expanduser("~/data/repos")
OUT_DIR = "rst_search"
DISALLOWED_TEXT = ["<|endoftext|>", ]

In [3]:
FILES_SUFFIX = [
    '.txt', '.md', '.py', '.ipynb',  '.js', '.html',
    '.java', '.c', '.cpp', '.h', '.hpp', '.sh', '.bash'
]


In [4]:
config_list_gpt3 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-35-turbo", "gpt-3.5-turbo"],
    },
)

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

gpt3_llm_config = {"config_list": config_list_gpt3, "cache_seed": 42}
gpt4_llm_config = {"config_list": config_list_gpt4, "cache_seed": 42}

In [5]:
def _random_file(root: str, suffix: list, ignore_regex: list) -> str:
    # Initialize an empty list to store the filenames that
    # match the suffix criteria
    matching_files = []

    # Traverse through the directory tree starting from the root
    for dirpath, dirnames, filenames in os.walk(root):
        for filename in filenames:
            # Check if the file has one of the defined suffixes
            if any(filename.lower().endswith(s.lower()) for s in suffix):
                # The file extension matches one of the suffixes
                dirpath = dirpath.replace("\\", "/")
                folder_paths = dirpath.split("/")
                if any(f.startswith(".") for f in folder_paths):
                    # Skip: the file is in a hidden folder
                    continue
                
                fname = os.path.join(dirpath, filename)
                
                # We search "ignore_regex" with the path name and file name, but not
                # the repo name.
                rel_name = os.path.relpath(fname, root)
                if any(re.search(regex, rel_name) for regex in ignore_regex):
                    # The file name matches one of the ignore regexes
                    continue
                # Append the full path of the file to the list
                matching_files.append(fname)
                
                
    if len(matching_files) == 0:
        pdb.set_trace()
    assert len(matching_files), "There is no matching file. Strange! Please take a look!"

    # Randomly select a file from the list of matching files
    return random.choice(matching_files)


In [6]:
def _random_block(filename: str, n_max_words: int) -> str:
    """
    Given a large file, return a random chunk of text (split by lines) 
    containing up to n_max_words words.
    
    Parameters:
        filename (str): The path to the file from which to read.
        n_max_words (int): The maximum number of words that the random block of 
            text should contain.
        
    Returns:
        str: A string containing the random block of text from the file.
    """
    # Initialize an empty list to store all lines from the file
    all_lines = []

    # Read all lines from the file and store them in the list
    with open(filename, 'r') as f:
        all_lines = f.readlines()

    # If the file is empty or contains no lines, return an empty string
    if not all_lines:
        return ""

    # Randomly choose a starting line index
    start_idx = random.randint(0, len(all_lines) - 1)

    # Initialize variables to keep track of the number of words and the
    # selected lines
    n_words = 0
    selected_lines = []

    # Loop to collect lines until n_max_words is reached or the end of
    # the file is reached
    for line in all_lines[start_idx:]:
        # line_words = line.split()
        n_words += num_tokens(line)

        if n_words > n_max_words:
            break

        selected_lines.append(line)

    return '\n'.join(selected_lines)



In [7]:
def norm_path(filename):
    # Check if the string contains any non-alphanumeric characters
    if any(not ch.isalnum() and ch not in [".", "_", "-", "/"]
           for ch in filename):
        return f'"{filename}"'
    return filename


def unnorm_path(filename):
    if filename[0] in ["\'", "\""] and filename[0] == filename[-1]:
        return unnorm_path(filename[1:-1])
    return filename


In [8]:
def commands_to_reach_destination(start: str,
                                  destination: str,
                                  folder_find_acc: float = 0.8) -> List[str]:
    """Use Linux's "ls", "cd", and "cat command to explore the path, 
    from `start` to the `destination`.

    Note that you are unfamiliar with the path, so you may need to "ls" to see
    the content inside a folder.

    It is guaranteed that the start and destination exist. 

    Args:
        start (str): a path
        destination (str): filename, a file which we want to find. 
        folder_find_acc (float, optional): the probability of finding a 
            correct folder. Defaults to 0.8.

    Returns:
        commands (list): a list of commands
    """
    assert os.path.isdir(start)
    assert os.path.isfile(destination)

    # Initialize an empty list to store the commands
    commands = []

    curr = start
    while True:
        commands.append("ls")

        # List the contents in the current directory
        contents = os.listdir(curr)

        # Separate the contents into files and folders
        files = [f for f in contents if os.path.isfile(os.path.join(curr, f))]
        folders = [f for f in contents if os.path.isdir(os.path.join(curr, f))]

        # Roll a dice to find the correct file
        if os.path.realpath(destination) in [
                os.path.realpath(os.path.join(curr, fname)) for fname in files
        ]:
            correct_command = f"cat {norm_path(os.path.basename(destination))}"
        else:
            # If we haven't found the file, we should try folders
            correct_path = os.path.relpath(destination, curr)
            cd_to = correct_path.split('/')[0]
            correct_command = f"cd {norm_path(cd_to)}"

        # List all possible actions
        all_possible_commands = [
            f"cat {norm_path(os.path.basename(fname))}" for fname in files
        ] + [f"cd {norm_path(os.path.basename(folder))}" for folder in folders]
        if curr != start:
            all_possible_commands += ["cd .."]

        if random.random() <= folder_find_acc:
            commands.append(correct_command)
        else:
            commands.append(random.choice(all_possible_commands))

        if commands[-1].startswith("cd "):
            dirname = unnorm_path(commands[-1][3:])
            curr = os.path.join(curr, dirname)
            curr = os.path.realpath(curr)
        else:
            cat_fname = unnorm_path(commands[-1].split()[-1])
            if os.path.realpath(os.path.join(
                    curr, cat_fname)) == os.path.realpath(destination):
                break

        if len(commands) > MAX_COMMAND_STEPS:
            raise RuntimeError("Too many commands")

    # Remove redundant "ls" command.
    # iterate through the commands, we don't need two adjacent "ls" commands,
    # even if there are some "cat" inside.
    rst = []
    need_ls = True
    for cmd in commands:
        if cmd.startswith("ls"):
            if need_ls:
                rst.append(cmd)
                need_ls = False
        else:
            rst.append(cmd)
            if cmd.startswith("cd"):
                need_ls = True

    print(colored(destination, "green"))
    print(commands)

    return commands


In [9]:

def optimal_path(start: str, destination: str) -> List[str]:
    """Use Linux's "ls", "cd", and "cat command to explore the path, 
    from `start` to the `destination`.

    Note that you are unfamiliar with the path, so you may need to "ls" to see
    the content inside a folder.

    It is guaranteed that the start and destination exist. 

    Args:
        start (str): a path
        destination (str): filename, a file which we want to find. 
        folder_find_acc (float, optional): the probability of finding a 
            correct folder. Defaults to 0.8.

    Returns:
        commands (list): a list of commands
    """
    assert os.path.isdir(start)
    assert os.path.isfile(destination)

    # folders = os.path.relpath(destination, start).split("/")
    # commands = []
    # for dirname in folders[:-1]:
    #     commands += ["ls", f"cd {dirname}"]
    # commands += ["ls", f"cat {folders[-1]}"]

    folders = os.path.relpath(destination, start).split("/")
    commands = [
        cmd for dirname in folders[:-1]
        for cmd in ["ls", f"cd {norm_path(dirname)}"]
    ] + ["ls", f"cat {norm_path(folders[-1])}"]

    return commands

In [10]:
def _random_question(root: str, filename: str, block: str, question_asker: AssistantAgent, user: UserProxyAgent) -> str:

    
    user.initiate_chat(question_asker, message=f"""
Now, ask questions regarding the following file.

--- Here are a part of the content from the file {filename} ---
{block}
------""")
    
    ans = user.last_message()["content"]
    
    pairs = re.findall(r'QUESTION:\s*(.*?)\nANSWER:\s*(.*?)\n\n', ans, re.DOTALL)
   
    if len(pairs) == 0:
        # Maybe just the answer is not generated. try again with "question" only.
        questions = re.findall(r'QUESTION:\s*(.*?)', ans, re.DOTALL)

        if len(questions) == 0:
            print(colored("NO questions generated by GPT!", "red"), filename)
            print(colored(ans, "yellow"))
            return []
        pairs = [(_ques, "") for _ques in questions]
        
    
    if not NEED_VERIFY:
        return pairs
    
    # Verify the generated question
    verified_pairs = []
    for question, answer in pairs:
        verify_prompt = f"""
Is the following question and answer specific to the given content? Or, is it a general question?

--- Here are the content ---
{block}

Question: {question}
Answer: {answer}

--- Reply with one word: SPECIFIC or GENERAL ---
"""
        ans_v = reply(verify_prompt, model_name=MODEL_NAME)

        if ans_v.upper().find("SPECIFIC") >= 0:
            verified_pairs.append((question, answer))
        else:
            print(colored("Skip the question:", "red"), question, "\t", answer)

    return verified_pairs


In [11]:
def gen(root: str, n_files: int = 10, outname: str = "out.json"):
    if os.path.exists(outname):
        print(f"Skip {outname}, because it already exists~")
        return

    rst = []
    num_success_attempts = 0
    
    architecture = list_files(root, file_suffixes=FILES_SUFFIX)
    architecture = architecture[:100] # use the first 100 files if there are too many.
    system_prompt = f"""
You are responsible to ask questions regarding a specific file inside a repository.

--- Repository Architecture ---
{architecture}
------


You need generate reading comprehension questions and answers based on the content.
You question should be very specific to the given file and the given content.
If the question is too general, don't even bother generating it.
The question should NOT be relevant to other files within the repository.

Use the format:

QUESTION: a question goes here
ANSWER: the answer to the question goes here

--- Good question example ---
QUESTION: How to generate an agent in the package?
QUESTION: Write code for "optimizer".
QUESTION: Show me suffix that are ignored from the pre-processing step.


--- Bad question example ---
QUESTION: What is the purpose of the code?
QUESTION: How to use the help function?
QUESTION: What is the purpose of @staticmethod?

"""
    open("prompt.tmp", "w").write(system_prompt)

    question_asker = AssistantAgent(
            name="Asker",
            human_input_mode="NEVER",
            max_consecutive_auto_reply=1,
            system_message=system_prompt,
            llm_config=gpt3_llm_config,
        )
    
    user = autogen.UserProxyAgent(
        name="User",
        human_input_mode="NEVER",
        max_consecutive_auto_reply=0
    )
    
    num_failed_attempts = 0
    while num_success_attempts < n_files and num_failed_attempts < 100:
        filename = _random_file(
            root=root,
            suffix=FILES_SUFFIX,
            ignore_regex=[".*out.*", ".*\.git.*", ".*test.*"])
        
        try:
            block = _random_block(filename, 3000)
        except Exception as e:
            print(e)
            num_failed_attempts += 1
            continue
        
        if num_tokens(block) < 100:
            print("A small block ignored in:", filename)
            num_failed_attempts += 1
            continue
        for disallowed in DISALLOWED_TEXT:
            if block.find(disallowed) >= 0:
                print(f"The string `{disallowed}` is in the block, which is disallowed!")
                num_failed_attempts += 1
                continue
        pairs = _random_question(root, filename, block, question_asker, user)

        if len(pairs) == 0:
            print(colored("NO questions generated by GPT!", "red"), filename)
        else:
            num_success_attempts += 1

        optimal = optimal_path(root, filename)  # the optimal path
        n_level = len([cmd for cmd in optimal if cmd.startswith("cd ")])

        for question, answer in pairs:
            try:
                commands = commands_to_reach_destination(root,
                                                         filename,
                                                         folder_find_acc=0.8)
            except Exception as e:
                print(e)
                continue

            print(colored(optimal, "red"))

            rst.append({
                "question": question,
                "answer": answer,
                "commands": commands,
                "optimal_path": optimal,
                "filename": os.path.relpath(filename, root),
                "root": os.path.relpath(root, REPOS_ROOT),
                "n_level": n_level
            })

    if len(rst) == 0:
        pdb.set_trace()

    # dump `rst` to json
    with open(outname, "w") as f:
        json.dump(rst, f, indent=2)



In [ ]:
if __name__ == "__main__":

    random.seed(1)

    # print(cmds)

    os.makedirs(OUT_DIR, exist_ok=True)
    for dirname in os.listdir(REPOS_ROOT):
        print(f"Processing {dirname}...")
        gen(os.path.join(REPOS_ROOT, dirname),
            n_files=10,
            outname=os.path.join(OUT_DIR, f"{dirname}.json"))

        # pdb.set_trace()

    # Combine all /*.json into one json file
    all_data = []
    for filename in glob.glob(os.path.join(OUT_DIR, "*.json")):
        all_data += json.load(open(filename, "r"))

    json.dump(all_data, open("file_search.json", "w"), indent=2)

Processing chatbot-ui-main...
Skip rst_search/chatbot-ui-main.json, because it already exists~
Processing consistency_models-main...
Skip rst_search/consistency_models-main.json, because it already exists~
Processing LongLoRA-main...
Skip rst_search/LongLoRA-main.json, because it already exists~
Processing 3x-ui-main...
Skip rst_search/3x-ui-main.json, because it already exists~
Processing sqlcoder-main...
Skip rst_search/sqlcoder-main.json, because it already exists~
Processing floatui-main...
Skip rst_search/floatui-main.json, because it already exists~
Processing ai-main...
Skip rst_search/ai-main.json, because it already exists~
Processing gpt-migrate-main...
Skip rst_search/gpt-migrate-main.json, because it already exists~
Processing baize-chatbot-main...
Skip rst_search/baize-chatbot-main.json, because it already exists~
Processing conc-main...
Skip rst_search/conc-main.json, because it already exists~
Processing lit-llama-main...
Skip rst_search/lit-llama-main.json, because it a


Now, ask questions regarding the following file.

--- Here are a part of the content from the file /home/beibinli/data/repos/TinyLlama-main/speculative_decoding/README.md ---
```

encoded   25 tokens in    0.278 seconds, speed:   89.900 t/s

decoded  258 tokens in    6.432 seconds, speed:   40.112 t/s



n_draft   = 28

n_predict = 258

n_drafted = 278

n_accept  = 200

accept    = 71.942%



draft:



llama_print_timings:        load time =   932.54 ms

llama_print_timings:      sample time =   583.50 ms /     1 runs   (  583.50 ms per token,     1.71 tokens per second)

llama_print_timings: prompt eval time =    81.50 ms /    25 tokens (    3.26 ms per token,   306.73 tokens per second)

llama_print_timings:        eval time =  1834.67 ms /   329 runs   (    5.58 ms per token,   179.32 tokens per second)

llama_print_timings:       total time =  6710.30 ms



target:



llama_print_timings:        load time = 18568.44 ms

llama_print_timings:      sample time =   208.78 ms /   258 r


--------------------------------------------------------------------------------
Asker (to User):

QUESTION: How can you obtain TinyLlama's evaluation scores on tasks such as hellaswag, openbookqa, winogrande, arc_easy, arc_challenge, boolq, and piqa?
ANSWER: You can obtain the evaluation scores by running lm-eval-harness with the specified model and tasks using the provided command.

QUESTION: How can you evaluate TinyLlama's ability in problem-solving on the Instruct-Eval evaluation suite?
ANSWER: You can evaluate TinyLlama's ability in problem-solving on the Instruct-Eval evaluation suite by running instruct-eval with the specified model name and model path using the provided commands.

--------------------------------------------------------------------------------
/home/beibinli/data/repos/TinyLlama-main/EVAL.md
['ls', 'cat EVAL.md']
['ls', 'cat EVAL.md']
A small block ignored in: /home/beibinli/data/repos/TinyLlama-main/scripts/prepare_slimpajama.py
A small block ignored in: /ho


--------------------------------------------------------------------------------
/home/beibinli/data/repos/TinyLlama-main/scripts/prepare_slimpajama.py
['ls', 'cd sft', 'ls', 'cd ..', 'ls', 'cd scripts', 'ls', 'cat prepare_starcoder.py', 'ls', 'cat convert_lit_checkpoint.py', 'ls', 'cat prepare_slimpajama.py']
['ls', 'cd scripts', 'ls', 'cat prepare_slimpajama.py']
User (to Asker):


Now, ask questions regarding the following file.

--- Here are a part of the content from the file /home/beibinli/data/repos/TinyLlama-main/README.md ---
## Finetune

We include a simple full-parameter finetuning & inference script in [sft](sft). Our V0.1 chat model is finetuned using this script. The FT dataset we use is [openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco). 

For finetuning with less than 4GB RAM, we refer you to the [Qlora](https://github.com/artidoro/qlora) and [bitsandbytes](https://github.com/TimDettmers/bitsandbytes) repo.

We did not undergone 


--------------------------------------------------------------------------------
Asker (to User):

QUESTION: How do you save the JavaScript code to a file inside the playground?
ANSWER: You can save the JavaScript code to a file inside the playground using the session.filesystem.write('/code/index.js', code) command.

QUESTION: How do you start the execution of the JavaScript file that is saved?
ANSWER: You can start the execution of the JavaScript file that is saved using the session.process.start() function, passing the command "node /code/index.js".

QUESTION: What actions are performed for streaming stdout and stderr during the execution of the JavaScript file?
ANSWER: During the execution of the JavaScript file, the program streams the stdout and stderr, and uses the on_stdout=print_out and on_stderr=print_out commands for handling the output.

--------------------------------------------------------------------------------
/home/beibinli/data/repos/e2b-main/apps/docs/src/code/py

...


--------------------------------------------------------------------------------
Asker (to User):

QUESTION: What is the purpose of the `on_validation_epoch_end` method in zero123.py?
ANSWER: The purpose of the `on_validation_epoch_end` method is to save an image sequence and remove the corresponding directory at the end of a validation epoch.

QUESTION: How does the `test_step` method in zero123.py handle the saving of image grids?
ANSWER: The `test_step` method in zero123.py saves image grids consisting of RGB images, normal maps, depth maps, and opacity maps by using the `save_image_grid` function with different types of images based on the output and batch data.

QUESTION: In zero123.py, how does the `test_step` method handle the saving of image sequences at the end of a test epoch?
ANSWER: The `test_step` method in zero123.py handles the saving of image sequences at the end of a test epoch by using the `save_img_sequence` function to save the image sequence and then remove the co


--------------------------------------------------------------------------------
Asker (to User):

QUESTION: What is the purpose of the `update_step` function in the file?
ANSWER: The `update_step` function is used to update the training steps, clip gradients for stable training, and adjust the minimum and maximum steps based on the training epoch and global step.

QUESTION: How is the `forward` function used in the file?
ANSWER: The `forward` function takes input parameters such as rgb, elevation, azimuth, camera distances, mvp_mtx, c2w, and other optional keyword arguments to perform operations including encoding images into latents with VAE, sampling timesteps, sampling noise, and calculating loss with different weighting strategies.

QUESTION: What is the purpose of the `F.mse_loss` function in the file?
ANSWER: The `F.mse_loss` function is used to calculate the mean squared error loss between the predicted noise and the target, with the option for reduction mean.

---------------


--------------------------------------------------------------------------------
Asker (to User):

QUESTION: What is the purpose of the function `create_vae_diffusers_config` in the script?
ANSWER: The purpose of the function `create_vae_diffusers_config` is to create a configuration for the diffusers based on the configuration of the LDM model. It takes the original configuration and an image size as input, and returns a configuration dictionary for the diffusers.

QUESTION: What does the function `convert_ldm_vae_checkpoint` do in the script?
ANSWER: The function `convert_ldm_vae_checkpoint` extracts the state dict for VAE from the checkpoint, processes the state dict, and assigns it to a new checkpoint with additional replacements, based on the provided configuration. It handles the conversion of a checkpoint from the LDM VAE model to the diffusers model.

--------------------------------------------------------------------------------
/home/beibinli/data/repos/threestudio-main/scr


--------------------------------------------------------------------------------
Asker (to User):

QUESTION: ¿Cuál es el propósito del comando `--query-token-pre TEXT` en los modelos transformadores?
ANSWER: El propósito del comando `--query-token-pre TEXT` es agregar un token a cada consulta en los modelos transformadores.

QUESTION: ¿Cuál es el propósito del comando `--annoy` en relación con el uso de kNN aproximados a través de Annoy para consultas?
ANSWER: El propósito del comando `--annoy` es determinar si se debe utilizar kNN aproximados a través de Annoy para consultas, lo cual puede permitir consultas más rápidas con un ligero costo de precisión.

QUESTION: ¿Dónde se encuentra la aplicación de Python para Semantra?
ANSWER: La aplicación de Python para Semantra se encuentra en `src/semantra/semantra.py`.

QUESTION: ¿Qué se debe ejecutar para compilar la aplicación web en modo de observación y reconstruirla cuando haya cambios?
ANSWER: Para compilar la aplicación web en modo de 


--------------------------------------------------------------------------------
